# Get backend information

## View backend information

In contrast to the [system configuration](#system-configuration-values) that remains static on the timescale of typical experiments, the **properties** of a system are dynamical quantities that update at system calibration time. Systems are nominally calibrated once over a 24-hour period, and the system properties update once this calibration sequence is complete. The properties play a critical role in quantum circuit execution, as the parameters within the properties are utilized for noise-aware circuit mapping and optimization (transpilation). Additionally, system properties are used for constructing noise models of a system for use in classical simulation methods.

You can most easily obtain a quantum system’s properties with Qiskit, but you can also obtain them by downloading the “calibrations” CSV file for a given system on IBM Quantum (see [View system configuration](#view-system-configuration) for more information).

*   **system name** - `system.properties().backend_name` gives the name of the system.

*   **system version** - `system.properties().backend_version` specifies the version of the system, as described [in the system versioning section](#system-versioning).

*   **last update date** - `system.properties().last_update_date` gives the local time at which the properties information was last updated. This is the time at which the full properties file was returned from the system. Individual elements in the properties give the times at which they were computed. Depending on the device calibration schedule, these times can be markedly different than the last update date.

*   **qubits** - `system.properties().qubits` contains information on the physical attributes of the qubits. This includes the qubit frequency in GHz, T1 and T2 times in µs, and readout error. As shown below, the readout\_error is further partitioned into `prob_meas0_prep1` and `prob_meas1_prep0`, indicating the probability of preparing a given computational basis state, but measuring the orthogonal state. The readout error is taken to be the average of these two values. Note that `prob_meas0_prep1` is usually the dominant factor in the readout error due to relaxation during the measurement process. The qubits field is a list indexed by the qubit number.

In [ ]:
[Nduv(datetime.datetime(2020, 12, 7, 9, 5, 37, tzinfo=tzlocal()), T1, us, 134.6122100394965),
Nduv(datetime.datetime(2020, 12, 7, 9, 6, 19, tzinfo=tzlocal()), T2, us, 69.51042482642832),
Nduv(datetime.datetime(2020, 12, 7, 9, 21, 20, tzinfo=tzlocal()), frequency, GHz, 4.743896496595851),
Nduv(datetime.datetime(2020, 12, 7, 9, 21, 20, tzinfo=tzlocal()), anharmonicity, GHz, -0.31386048358781926),
Nduv(datetime.datetime(2020, 12, 7, 9, 4, 47, tzinfo=tzlocal()), readout_error, , 0.06969999999999998),
Nduv(datetime.datetime(2020, 12, 7, 9, 4, 47, tzinfo=tzlocal()), prob_meas0_prep1, , 0.0796),
Nduv(datetime.datetime(2020, 12, 7, 9, 4, 47, tzinfo=tzlocal()), prob_meas1_prep0, , 0.0598),
Nduv(datetime.datetime(2020, 12, 7, 9, 4, 47, tzinfo=tzlocal()), readout_length, ns, 5813.333333333333)
]

We see that each qubit parameter is expressed as an `Nduv` (name, date, unit value) object containing the local time at which the parameter was updated, the parameter name, parameter units, and the actual numerical parameter value.

*   **gates** - `system.properties().gates` gives detailed information on each gate that the system supports executing. For every single-qubit gate listed in the system basis gates, there exists an entry in the properties that gives the details of that gate, for each qubit on which the gate can be applied. Each entry is in the form of a `Gate` object:

In [ ]:
Gate(gate='u2', name='u2_0', parameters=[Nduv('2020-06-06T09:06:46Z', gate_error, , 0.00022436814887248838), Nduv('2020-06-06T10:03:18Z', gate_length, ns, 35.55555555555556)], qubits=[0])

and lists the kind of gate (`u2` in the example), its name, a list specifying the qubits on which the gate acts, and a `parameters` attribute, `gates.parameters`:

In [ ]:
[Nduv('2020-06-06T09:06:46Z', gate_error, , 0.00022436814887248838),
Nduv('2020-06-06T10:03:18Z', gate_length, ns, 35.55555555555556)]

        containing `Nduv` objects specifying the gate error and gate length.

        The same holds for multi-qubit gates, where there is an entry for each directed edge in the system [coupling map](#coupling-map). For symmetric coupling maps, the directionality of the underlying gate is still important. For example,

In [ ]:
Gate(gate='cx', name='cx3_4', parameters=[Nduv('2020-06-06T10:03:18Z', gate_error, , 0.009793933588297388), Nduv('2020-06-06T10:03:18Z', gate_length, ns, 327.1111111111111)], qubits=[3, 4])

versus

In [ ]:
Gate(gate='cx', name='cx4_3', parameters=[Nduv('2020-06-06T10:03:18Z', gate_error, , 0.009793933588297388), Nduv('2020-06-06T10:03:18Z', gate_length, ns, 362.66666666666663)], qubits=[4, 3])

## Configuration

The configuration is where you'll find data about the static setup of the device, such as its name, version, the number of qubits, and the types of features it supports.

Let's build a description of our backend using information from the `backend`'s config.

In [1]:
from qiskit.providers.fake_provider import FakeHanoi

backend = FakeHanoi()

In [2]:
config = backend.configuration()

# Basic Features
print("This backend is called {0}, and is on version {1}. It has {2} qubit{3}. It "
      "{4} OpenPulse programs. The basis gates supported on this device are {5}."
      "".format(config.backend_name,
                config.backend_version,
                config.n_qubits,
                '' if config.n_qubits == 1 else 's',
                'supports' if config.open_pulse else 'does not support',
                config.basis_gates))

This backend is called fake_hanoi, and is on version 1.0.18. It has 27 qubits. It supports OpenPulse programs. The basis gates supported on this device are ['id', 'rz', 'sx', 'x', 'cx', 'reset'].


## Properties

The `backend` properties contain data that was measured and optionally reported by the provider. Let's see what kind of information is reported for qubit 0.

In [ ]:
props = backend.properties()

In [ ]:
def describe_qubit(qubit, properties):
    """Print a string describing some of reported properties of the given qubit."""

    # Conversion factors from standard SI units
    us = 1e6
    ns = 1e9
    GHz = 1e-9

    print("Qubit {0} has a \n"
          "  - T1 time of {1} microseconds\n"
          "  - T2 time of {2} microseconds\n"
          "  - U2 gate error of {3}\n"
          "  - U2 gate duration of {4} nanoseconds\n"
          "  - resonant frequency of {5} GHz".format(
              qubit,
              properties.t1(qubit) * us,
              properties.t2(qubit) * us,
              properties.gate_error('sx', qubit),
              properties.gate_length('sx', qubit) * ns,
              properties.frequency(qubit) * GHz))

describe_qubit(0, props)

Qubit 0 has a 
  - T1 time of 162.29562357444243 microseconds
  - T2 time of 171.74648699183206 microseconds
  - U2 gate error of 0.00013790682762652163
  - U2 gate duration of 21.333333333333332 nanoseconds
  - resonant frequency of 5.035257503599211 GHz
